In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import beta 
from scipy.stats import binomtest 

from scipy.stats import ks_2samp

# library for adjusting p-values for multiple hypothesis testing where the Benjamini-Hockberg procedure is standard 
from scipy.stats import false_discovery_control

# Description: 

In this notebook we simulate data to analyse the distance measure. Note that we perform hypothesis test but we are only interested in the p-values to derive a distance measure from those. 

## Binomial test (exact test)

We assume that $X \sim Bin(\theta_0, m)$ and test the hypothesis that $H_0:\theta > \theta_0$, Thus, the p-value is given by 
\begin{align*}
p = \sum_{i=k}^m P(X=i) = \sum_{i=k}^m \binom{m}{k} \theta_0^i(1-\theta_0)^{m-i}. 
\end{align*}

## Z-test (approximate test)

For large samples we can use the Z-test. Thus, we assume that $Z\sim N(0, 1)$ and then the test statistic is given by 

\begin{align*}
z_{obs} = \frac{\hat{\theta}-\theta_0}{\sqrt{\frac{\theta_0(1-\theta_0)}{m}}}. 
\end{align*}

The p-value is then for a left one sided test given by $p = P(Z \geq z_{obs})$ ect. vice versa for a right one sided test and for the two-sided test the p-value is $p=2\cdot(1-P(Z\leq z_{obs}))$. For the two-sided test we use the symmetry of the normal distribution which implies that we can double the p-value.   

Questions that arises: 

- Note for both tests the p-value is dependent of the threshold $\theta_0$ - How do we choose the threshold value $\theta_0$? 
- To derive a distance measure we use that small p-values corresponds to large distances - how do we derive a good distance measure? 
We now have the following distance measure: 
\begin{align*}
d(A,B) = \frac{1}{n}\sum_{i=1}^n |M_{B,i} - M_{A,i}| \in [0,1]
\end{align*}
where $M_{A,i} = 1[p_{Ai}>\alpha]$ and  $M_{B,i} = 1[p_{Bi}>\alpha]$ or we just let $M_{Ai}$ and $M_{Bi}$ equal the corresponding p-values. 

- How large are the samples $\alpha_{N}+\beta_{N}$? (scale and shape paremeters of the posterior) 

Idears: 
 - Is it better to use a non-parametric (no assumption of distributions) like the KS-test to avoid setting a threshold or can we use that the mythylating rate greater than $\theta=0.5$?
 - Note the p values corresponds to if the correlation is significantly different from 0. Another posibility is to use the correlation between to binary vectors, we could use the Phi corelation here, and then derive a distrance measure from the corelation instead of the p-values? The distance could for instance be $d(A,B)=1-|\rho(X,Y)|$ where $X$ and $Y$ both follows a binormail distribution, (not pair this does not work...)

In [ ]:
# succes probabilities 
theta_Ai = [0.9, 0.8, 0.6]; theta_Bi = [0.6, 0.1, 0.6]

# note we use the same succesprobabilities for contig A and B

# Test on simulated data 

In [ ]:
def sim_contigsep(theta_Ai, theta_Bi, m, theta_0=0.5, d_fun = ["p", "p_binary"]):

    sim_A = np.array([np.random.binomial(1, Ai, m) for Ai in theta_Ai]) 

    sim_B = np.array([np.random.binomial(1, Bi, m) for Bi in theta_Bi]) 

    M_A = np.array([binomtest(k=np.sum(sim_Ai), n=m, p=theta_0, alternative="less").pvalue for sim_Ai in sim_A])

    M_B = np.array([binomtest(k=np.sum(sim_Bi), n=m, p=theta_0, alternative="less").pvalue for sim_Bi in sim_B])


    n = M_A.shape[0]

    if(n > 1): 
        M_A = false_discovery_control(M_A)
        M_B = false_discovery_control(M_B)


    M_A_binary = np.where(M_A>0.05, 1, 0)
    M_B_binary = np.where(M_B>0.05, 1, 0)

    if(d_fun == "p"):
        d = 1/n*np.sum(np.abs(M_A-M_B)) 

    elif(d_fun == "p_binary"): 
        d = 1/n*np.sum(np.abs(M_A_binary-M_B_binary))
  
    return d

# Boxplots of distance d with different number of motifs m 

In [ ]:
d_5 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 5, d_fun ="p") for i in range(0, 1000)])
d_10 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 10, d_fun ="p") for i in range(0, 1000)])
d_15 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 15, d_fun ="p") for i in range(0, 1000)])
d_30 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 30, d_fun ="p") for i in range(0, 1000)])
d_40 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 40, d_fun ="p") for i in range(0, 1000)])
d_50 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 50, d_fun ="p") for i in range(0, 1000)])
d_100 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 100, d_fun ="p") for i in range(0, 1000)])
d_500 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 500, d_fun ="p") for i in range(0, 1000)])
d_1000 = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 1000, d_fun ="p") for i in range(0, 1000)])

d_dict = {"5": d_5, "10": d_10, "15": d_15, "30": d_30, "40": d_40, "50": d_50, "100": d_100, "500": d_500, "1000": d_1000}

fig, ax = plt.subplots()
ax.boxplot(d_dict.values())
ax.set_xticklabels(d_dict.keys())
plt.title("Boxplot of p-value distance for different value of m")
plt.ylabel("d")
plt.xlabel("m")
plt.show()

In [ ]:
d_5_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 5, d_fun ="p_binary") for i in range(0, 1000)])
d_10_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 10, d_fun ="p_binary") for i in range(0, 1000)])
d_15_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 15, d_fun ="p_binary") for i in range(0, 1000)])
d_30_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 30, d_fun ="p_binary") for i in range(0, 1000)])
d_40_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 40, d_fun ="p_binary") for i in range(0, 1000)])
d_50_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 50, d_fun ="p_binary") for i in range(0, 1000)])
d_100_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 100, d_fun ="p_binary") for i in range(0, 1000)])
d_500_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 500, d_fun ="p_binary") for i in range(0, 1000)])
d_1000_binary = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m= 1000, d_fun ="p_binary") for i in range(0, 1000)])

d_dict = {"5": d_5_binary, "10": d_10_binary, "15": d_15_binary, "30": d_30_binary, "40": d_40_binary, "50": d_50_binary, "100": d_100_binary, "500": d_500_binary, "1000": d_1000_binary}

fig, ax = plt.subplots()
ax.boxplot(d_dict.values())
ax.set_xticklabels(d_dict.keys())
plt.title("Boxplot of binary distance for different value of m")
plt.ylabel("d")
plt.xlabel("m")
plt.show()

# Plot of distance d against m 

In [ ]:
size_m = np.arange(5, 1005, 5)

d1_sample = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m = mi, d_fun ="p") for mi in size_m])
d2_sample = np.array([sim_contigsep(theta_Ai=theta_Ai, theta_Bi=theta_Bi, m = i, d_fun ="p_binary") for i in size_m])

# plots distance measures  

fig = plt.figure()

ax = fig.add_axes([0,0,1,1])

ax.plot(size_m, d1_sample, label="P-value distance measure")
ax.plot(size_m, d2_sample, label="Binary distance measure")
ax.legend()

plt.title("Distance measures against sample size")
plt.xlabel("m")
plt.ylabel("d")
plt.show()

# Two sample Kolmogorov–Smirnov test 

The Kolmogorov-Smirnov test is a non-parametric test meaning that we have no distribution assumptions. 

Assume $f$ and $g$ are continuous distributions, $F$ and $G$ corresponding CDF's and $F_n$ and $G_m$ empirical CDFs for $F$ and $G$, respectively.
Let $H_0: F=G$ and $H_A:F\neq G$ then the test statistics $D$ for the two-sample Kolmogorov-Smirnov test is given by 
\begin{align*}
D_{n,m}=\sup_{x}\left(| F_n(x)-G_m(x) |\right).  
\end{align*}

In this setup we can compare the posterior beta distributions for contig $A$ and$B$. We could use the p-value to derive a measure such that the distance $d$ is given by 
\begin{align*}
d(A,B) = \frac{1}{n}\cdot\sum_{i=1}^n M_{AB_i}
\end{align*}
where $M_{AB_i}$ is equal to the p-value obtained from a two sample Kolmogorov-Smirnov test for motif $i$ on contig $A$ and $B$. Note that here $d=1$ means that all the p-values are one and we do not reject $H_0$. 

## Simulated data

In [ ]:
m = 100 

sim_A = np.array([np.random.binomial(1, Ai, m) for Ai in theta_Ai]) 

sim_B = np.array([np.random.binomial(1, Bi, m) for Bi in theta_Bi]) 

alpha_N = np.sum(sim_A[0]); beta_N = sim_A[0].shape[0]-alpha_N 

s1 = np.sort(np.random.beta(a=alpha_N, b=beta_N, size = 100)) 
s2 = np.sort(np.random.beta(a=alpha_N, b=beta_N, size = 100)) 

ks_2samp(s1, s2).pvalue


In [ ]:
m = 100

sim_A = np.array([np.random.binomial(1, Ai, m) for Ai in theta_Ai]) 

sim_B = np.array([np.random.binomial(1, Bi, m) for Bi in theta_Bi]) 

s1 = np.array([np.sort(np.random.beta(a=np.sum(sim_Ai), b=(sim_Ai.shape[0]-np.sum(sim_Ai)), size = 100)) for sim_Ai in sim_A])

s2 = np.array([np.sort(np.random.beta(a=np.sum(sim_Bi), b=(sim_Bi.shape[0]-np.sum(sim_Bi)), size = 100)) for sim_Bi in sim_B])

M_AB = np.array([ks_2samp(s1_Ai, s2_Bi).pvalue for (s1_Ai, s2_Bi) in zip(s1, s2)])

M_AB = np.array([ks_2samp(s1_Ai, s2_Bi).pvalue for (s1_Ai, s2_Bi) in zip(s1, s2)])

M_AB

There is a mistake there values should be big - we should not reject $H_0$ in this case. 

In [ ]:
def KS_contigsep(theta_Ai, theta_Bi, m, d_fun = ["p", "p_binary"]):

    sim_A = np.array([np.random.binomial(1, Ai, m) for Ai in theta_Ai]) 

    sim_B = np.array([np.random.binomial(1, Bi, m) for Bi in theta_Bi]) 

    s1 = np.array([np.sort(np.random.beta(a=np.sum(sim_Ai), b=sim_Ai.shape[0]-np.sum(sim_Ai), size = 100)) for sim_Ai in sim_A])

    s2 = np.array([np.sort(np.random.beta(a=np.sum(sim_Bi), b=sim_Bi.shape[0]-np.sum(sim_Bi), size = 100)) for sim_Bi in sim_B])

    M_AB = np.array([ks_2samp(s1_Ai, s2_Bi).pvalue for s1_Ai, s2_Bi in zip(s1, s2)])


    n = M_AB.shape[0]
   
    if(n > 1): 
         M_AB = false_discovery_control(M_AB)
        
    M_AB_binary = np.where(M_AB>0.05, 1, 0)

    if(d_fun == "p"):
        d = 1/n*np.sum(M_AB) 

    elif(d_fun == "p_binary"): 
        d = 1/n*np.sum(M_AB_binary)
    
    return d